In [19]:
# Random Forest Classification with RandomizedsearchCV
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [20]:
# Import dataset
dataset = pd.read_csv('Social_Network_Ads.csv')
X = dataset.iloc[:,[2,3]].values
y = dataset.iloc[:,4].values

In [21]:
# Splitting the dataset on training set and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25, random_state = 0)

In [23]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
# Fitting Random Forest classifier on the training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=50)
classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=50, verbose=0,
                       warm_start=False)

In [25]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [26]:
est = RandomForestClassifier(n_jobs=-1)
rf_p_dist ={'max_depth':[3,5,10,None],
            'n_estimators':[10,100,200,300,400,500],
            'max_features':randint(1,3),
            'criterion':['gini','entropy'],
            'bootstrap':[True,False],
            'min_samples_leaf':randint(1,4) 
}

In [27]:
def hypertuning_rscv(est,p_dist,nbr_iter,X,y):
    rdmsearch = RandomizedSearchCV(est,param_distributions=p_dist,
                                    n_jobs=-1, n_iter=nbr_iter, cv=9)
    #CV = cross-validation (here we are using stratified KFold CV)
    rdmsearch.fit(X,y)
    ht_params = rdmsearch.best_params_
    ht_score = rdmsearch.best_score_
    return ht_params,ht_score

In [28]:
rf_parameters, rf_ht_score = hypertuning_rscv(est, rf_p_dist, 40, X, y)

In [29]:
print(rf_parameters,rf_ht_score)

{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'max_features': 2, 'min_samples_leaf': 1, 'n_estimators': 100} 0.9125


In [30]:
classifier = RandomForestClassifier(n_jobs=-1, n_estimators=300, bootstrap= True,criterion='entropy', max_depth=3, max_features=2, min_samples_leaf= 3 )

In [32]:
# Predicting the test set results
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

In [38]:
# Making the confusion matrix
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
accuracy_score = accuracy_score(y_test, y_pred)

In [39]:
# Cross validation good for selecting the models
from sklearn.model_selection import cross_val_score
cross_val = cross_val_score(classifier,X,y,cv=10,scoring='accuracy').mean()